In [1]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

import pandas as pd
from MatrixVectorizer import *
import torch
import random
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
import optuna

In [2]:
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# Check for CUDA (GPU support) and set device accordingly
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # For multi-GPU setups
    # Additional settings for ensuring reproducibility on CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(device)

mps


In [3]:
# A_LR_train = pd.read_csv("../data/lr_train.csv")
# A_HR_train = pd.read_csv("../data/hr_train.csv")
# A_LR_test = pd.read_csv("../data/lr_test.csv")

In [4]:
LR_size = 160
HR_size = 268

In [5]:
# from sklearn.decomposition import PCA
# from sklearn.mixture import GaussianMixture

# A_HR_train = pd.read_csv("../data/hr_train.csv")

# pca = PCA(n_components=0.99, whiten=False)
# A_HR_train_pca = pca.fit_transform(A_HR_train)
# print(A_HR_train_pca.shape)

# gm = GaussianMixture(n_components=5, random_state=random_seed)
# A_HR_train_label = gm.fit_predict(A_HR_train_pca)
# unique, counts = np.unique(A_HR_train_label, return_counts=True)
# print(np.asarray((unique, counts)).T)

# X = np.load('A_LR_train_matrix.npy')
# y = np.load('A_HR_train_matrix.npy')

# n_sample = X.shape[0]
# X_train, X_val, y_train, y_val = train_test_split(
#     X.reshape(n_sample, -1), 
#     y.reshape(n_sample, -1), 
#     test_size=0.20, 
#     random_state=random_seed,
#     stratify=A_HR_train_label
# )

# X_train = X_train.reshape(-1, LR_size, LR_size)
# X_val = X_val.reshape(-1, LR_size, LR_size)
# y_train = y_train.reshape(-1, HR_size, HR_size)
# y_val = y_val.reshape(-1, HR_size, HR_size)

# print("Train size:", len(X_train))
# print("Val size:", len(X_val))

In [6]:
# def antivectorize_df(adj_mtx_df, size):
    
#     num_subject = adj_mtx_df.shape[0]
#     adj_mtx = np.zeros((num_subject, size, size)) #torch.zeros((num_subject, LR_size, LR_size))
#     for i in range(num_subject):
#         adj_mtx[i] = MatrixVectorizer.anti_vectorize(adj_mtx_df.iloc[i], size) # torch.from_numpy(MatrixVectorizer.anti_vectorize(A_LR_train.iloc[i], LR_size))
#     return adj_mtx

# np.save('A_LR_train_matrix.npy', antivectorize_df(A_LR_train, LR_size))
# np.save('A_HR_train_matrix.npy', antivectorize_df(A_HR_train, HR_size))
# np.save('A_LR_test_matrix.npy', antivectorize_df(A_LR_test, LR_size))

In [7]:
A_LR_train_matrix = np.load('A_LR_train_matrix.npy')
A_HR_train_matrix = np.load('A_HR_train_matrix.npy')
A_LR_test_matrix = np.load("A_LR_test_matrix.npy")

print(A_LR_train_matrix.shape)
print(A_HR_train_matrix.shape)
print(A_LR_test_matrix.shape)

(167, 160, 160)
(167, 268, 268)
(112, 160, 160)


In [8]:
import torch
import numpy as np
import torch.optim as optim
from sklearn.model_selection import KFold
from preprocessing import *
from model import *
from train import *
import argparse



epochs = 200


parser = argparse.ArgumentParser(description='GSR-Net')
parser.add_argument('--epochs', type=int, default=epochs, metavar='no_epochs',
                help='number of episode to train ')
parser.add_argument('--lr', type=float, default=0.0001, metavar='lr',
                help='learning rate (default: 0.0001 using Adam Optimizer)')
parser.add_argument('--splits', type=int, default=3, metavar='n_splits',
                help='no of cross validation folds')
parser.add_argument('--lmbda', type=int, default=16, metavar='L',
                help='self-reconstruction error hyperparameter')
parser.add_argument('--lr_dim', type=int, default=LR_size, metavar='N',
                help='adjacency matrix input dimensions')
parser.add_argument('--hr_dim', type=int, default=HR_size, metavar='N',
                help='super-resolved adjacency matrix output dimensions')
parser.add_argument('--hidden_dim', type=int, default=268, metavar='N',
                help='hidden GraphConvolutional layer dimensions')
parser.add_argument('--padding', type=int, default=26, metavar='padding',
                help='dimensions of padding')
parser.add_argument('--embedding_size', type=int, default=32, metavar='embedding_size',
                help='node embedding size')
parser.add_argument('--early_stop_patient', type=int, default=10, metavar='early_stop_patient',
                help='early_stop_patience')
parser.add_argument('--dropout_rate', type=float, default=0.2, metavar='dropout_rate',
                help='dropout_rate')

parser.add_argument('--p_perturbe', type=float, default=0.5, metavar='p_perturbe',
                help='p_perturbe')
parser.add_argument('--p_drop_node', type=float, default=0.03, metavar='p_drop_node',
                help='p_drop_node')
parser.add_argument('--p_drop_edges', type=float, default=0.1, metavar='p_drop_edges',
                help='p_drop_edges')
parser.add_argument('--mean_dense', type=float, default=0., metavar='mean',
                        help='mean of the normal distribution in Dense Layer')
parser.add_argument('--std_dense', type=float, default=0.01, metavar='std',
                    help='standard deviation of the normal distribution in Dense Layer')
parser.add_argument('--mean_gaussian', type=float, default=0., metavar='mean',
                    help='mean of the normal distribution in Gaussian Noise Layer')
parser.add_argument('--std_gaussian', type=float, default=0.1, metavar='std',
                    help='standard deviation of the normal distribution in Gaussian Noise Layer')


# Create an empty Namespace to hold the default arguments
args = parser.parse_args([]) 
print(args)

Namespace(epochs=200, lr=0.0001, splits=3, lmbda=16, lr_dim=160, hr_dim=268, hidden_dim=268, padding=26, embedding_size=32, early_stop_patient=10, dropout_rate=0.2, p_perturbe=0.5, p_drop_node=0.03, p_drop_edges=0.1, mean_dense=0.0, std_dense=0.01, mean_gaussian=0.0, std_gaussian=0.1)


In [9]:
# SIMULATING THE DATA: EDIT TO ENTER YOUR OWN DATA
X = A_LR_train_matrix #np.random.normal(0, 0.5, (167, 160, 160))
Y = A_HR_train_matrix #np.random.normal(0, 0.5, (167, 288, 288))
print(X.shape)
print(Y.shape)

(167, 160, 160)
(167, 268, 268)


In [10]:
device = get_device()
print(device)

mps


In [11]:
def compute_degree_matrix_normalization_batch_numpy(adjacency_batch):
    """
    Optimizes the degree matrix normalization for a batch of adjacency matrices using NumPy.
    Computes the normalized adjacency matrix D^-1 * A for each graph in the batch.
    
    Parameters:
    - adjacency_batch: A NumPy array of shape (batch_size, num_nodes, num_nodes) representing
                       a batch of adjacency matrices.

    Returns:
    - A NumPy array of normalized adjacency matrices.
    """
    epsilon = 1e-6  # Small constant to avoid division by zero
    # Calculate the degree for each node in the batch
    d = adjacency_batch.sum(axis=2) + epsilon
    
    # Compute the inverse degree matrix D^-1 for the batch
    D_inv = np.reciprocal(d)[:, :, np.newaxis] * np.eye(adjacency_batch.shape[1])[np.newaxis, :, :]
    
    # Normalize the adjacency matrix using batch matrix multiplication
    normalized_adjacency_batch = np.matmul(D_inv, adjacency_batch)
    
    return normalized_adjacency_batch
X = compute_degree_matrix_normalization_batch_numpy(X)
A_LR_test_matrix = compute_degree_matrix_normalization_batch_numpy(A_LR_test_matrix)
print(X.shape)

(167, 160, 160)


In [12]:
import gc

best_mae = 1000
best_params = None
best_model = None

def hyperparameter_search(n_trials=40):
    def objective(trial):
        global best_mae
        global best_params
        global best_model
        ks = [0.9, 0.7, 0.6, 0.5]
        num_epochs = 200
        lr = trial.suggest_float('lr', 9e-5, 3e-4, log=True)
        lmbda = trial.suggest_int('lmbda', 12, 52, step=4)
        # ks = trial.suggest_categorical('ks', [
        #     (0.9, 0.7, 0.6, 0.5), 
        #     (0.8, 0.5), 
        #     (0.75, 0.5, 0.25), 
        #     (0.9, 0.8, 0.7, 0.6, 0.5), 
        #     (0.5, ),
        #     (0.5, 0.25),
        # ])
        
        hidden_dim = 268
        stop_gan_mae = 0.135
        patience = 5
        dropout_rate = trial.suggest_float('dropout_rate', 0.05, 0.3, step=0.05)
        p_perturbe = trial.suggest_float('p_perturbe', 0, 0.7, step=0.05)
        p_drop_node = trial.suggest_float('p_drop_node', 0.01, 0.1, step=0.005)
        p_drop_edges = trial.suggest_float('p_drop_edges', 0.03, 0.18, step=0.005)

        args.epochs = num_epochs
        args.lr = lr
        args.lmbda = lmbda
        args.hidden_dim = hidden_dim
        args.early_stop_patient = patience
        args.dropout_rate = dropout_rate
        args.p_perturbe = p_perturbe
        args.p_drop_node = p_drop_node
        args.p_drop_edges = p_drop_edges

        params = {
                "lr": lr,
                "lmbda": lmbda,
                "hidden_dim": hidden_dim,
                "patience": patience,
                "dropout_rate": dropout_rate,
                "p_perturbe": p_perturbe,
                "p_drop_node": p_drop_node,
                "p_drop_edges": p_drop_edges,
                "ks": ks,
            }
        print(f"Trial {trial.number}: {params}")

        cv = KFold(n_splits=args.splits, random_state=random_seed, shuffle=True)

        mae_list = []
        i = 1
        for train_index, test_index in cv.split(X):
            torch.cuda.empty_cache()
            gc.collect()
            # print(f"----- Fold {i} -----")
            subjects_adj, test_adj, subjects_ground_truth, test_ground_truth = X[
                train_index], X[test_index], Y[train_index], Y[test_index]

            modelG = GSRNet(ks, args).to(device)
            optimizerG = torch.optim.Adam(modelG.parameters(), lr=args.lr)

            modelD = Discriminator(args).to(device)
            optimizerD = optim.Adam(modelD.parameters(), lr=args.lr)
 
            # return_model = train(modelG, optimizerG, subjects_adj, subjects_ground_truth, args, test_adj, test_ground_truth)
            return_model = train_gan(
                modelG, 
                optimizerG, 
                modelD, 
                optimizerD, 
                subjects_adj, 
                subjects_ground_truth, 
                args, 
                test_adj, 
                test_ground_truth,
                stop_gan_mae=stop_gan_mae
            )
            test_mae = test(return_model, test_adj, test_ground_truth, args)
            mae_list.append(test_mae)
            break

            i += 1

        mean_mae = np.mean(mae_list)
        if mean_mae < best_mae:
            best_mae = mean_mae
            best_params = params
            best_model = return_model
            print(f"New best MAE: {test_mae} with params: {best_params}")

        return mean_mae
    
    study = optuna.create_study(direction='minimize', study_name='GSR_hyperparameter_search', sampler=optuna.samplers.TPESampler())
    study.optimize(objective, n_trials=n_trials)
    return study.best_params

# best_params = hyperparameter_search(40)
# print(best_params)

In [13]:
ks = [0.9, 0.7, 0.6, 0.5]
gan_params = {
    'lr': 0.00010436239086997339, 
    'lmbda': 24, 
    'hidden_dim': 268, 
    'patience': 5, 
    'dropout_rate': 0.15000000000000002, 
    'p_perturbe': 0.5, 
    'p_drop_node': 0.05, 
    'p_drop_edges': 0.11
}

vars(args).update(gan_params)
print(args)

Namespace(epochs=200, lr=0.00010436239086997339, splits=3, lmbda=24, lr_dim=160, hr_dim=268, hidden_dim=268, padding=26, embedding_size=32, early_stop_patient=10, dropout_rate=0.15000000000000002, p_perturbe=0.5, p_drop_node=0.05, p_drop_edges=0.11, mean_dense=0.0, std_dense=0.01, mean_gaussian=0.0, std_gaussian=0.1, patience=5)


In [14]:
cv = KFold(n_splits=args.splits, random_state=random_seed, shuffle=True)

ks = [0.9, 0.7, 0.6, 0.5]
stop_gan_mae = 0.135

best_model_fold_list = []
data_fold_list = []
i = 1
for train_index, test_index in cv.split(X):

    print(f"----- Fold {i} -----")
    subjects_adj, test_adj, subjects_ground_truth, test_ground_truth = X[
        train_index], X[test_index], Y[train_index], Y[test_index]
    data_fold_list.append((subjects_adj, test_adj, subjects_ground_truth, test_ground_truth))

    netG = GSRNet(ks, args).to(device)
    optimizerG = optim.Adam(netG.parameters(), lr=args.lr)

    netD = Discriminator(args).to(device)
    optimizerD = optim.Adam(netD.parameters(), lr=args.lr)

    #GAN model
    return_model = train_gan(
        netG, 
        optimizerG, 
        netD,
        optimizerD,
        subjects_adj, 
        subjects_ground_truth, 
        args, 
        test_adj=test_adj, 
        test_ground_truth=test_ground_truth,
        # stop_gan_mae=stop_gan_mae
    )


    # To be delete
    # optimizerG = optim.Adam(return_model.parameters(), lr=args.lr)

    #Non-GAN model
    # return_model = train(netG, optimizerG, subjects_adj, subjects_ground_truth, args, test_adj, test_ground_truth)

    test_mae = test(return_model, test_adj, test_ground_truth, args)
    print(f"Val MAE: {test_mae}")
    best_model_fold_list.append(return_model)

    i += 1

    # break

----- Fold 1 -----


Epoch Progress:   0%|          | 0/200 [00:00<?, ?epoch/s]/Users/carlosbrat/Library/Mobile Documents/com~apple~CloudDocs/Imperial/Term2/Deep Graph-based Learning/CW2/graph_super_resolution/gsr_net/layers.py:35: UserWarning: The operator 'aten::_linalg_eigh.eigenvalues' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  eig_val_lr, U_lr = torch.linalg.eigh(lr, UPLO='U')
Epoch Progress:   0%|          | 1/200 [00:11<38:08, 11.50s/epoch, test_error=0.239, train_error=0.249, train_loss=2.61]


KeyboardInterrupt: 

In [ ]:
from MatrixVectorizer import MatrixVectorizer

from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
from scipy.spatial.distance import jensenshannon
import torch
import networkx as nx

def evaluate(pred_matrices, gt_matrices):

    num_test_samples = gt_matrices.shape[0]

    # Initialize lists to store MAEs for each centrality measure
    mae_bc = []
    mae_ec = []
    mae_pc = []

    pred_1d = []
    gt_1d = []
    for i in range(num_test_samples):
        pred_1d.append(MatrixVectorizer.vectorize(pred_matrices[i]))
        gt_1d.append(MatrixVectorizer.vectorize(gt_matrices[i]))


    # # Iterate over each test sample
    # for i in range(num_test_samples):
    #     # Convert adjacency matrices to NetworkX graphs
    #     pred_graph = nx.from_numpy_array(pred_matrices[i], edge_attr="weight")
    #     gt_graph = nx.from_numpy_array(gt_matrices[i], edge_attr="weight")

    #     # Compute centrality measures
    #     pred_bc = nx.betweenness_centrality(pred_graph, weight="weight")
    #     pred_ec = nx.eigenvector_centrality(pred_graph, weight="weight")
    #     pred_pc = nx.pagerank(pred_graph, weight="weight")

    #     gt_bc = nx.betweenness_centrality(gt_graph, weight="weight")
    #     gt_ec = nx.eigenvector_centrality(gt_graph, weight="weight")
    #     gt_pc = nx.pagerank(gt_graph, weight="weight")

    #     # Convert centrality dictionaries to lists
    #     pred_bc_values = list(pred_bc.values())
    #     pred_ec_values = list(pred_ec.values())
    #     pred_pc_values = list(pred_pc.values())

    #     gt_bc_values = list(gt_bc.values())
    #     gt_ec_values = list(gt_ec.values())
    #     gt_pc_values = list(gt_pc.values())

    #     # Compute MAEs
    #     mae_bc.append(mean_absolute_error(pred_bc_values, gt_bc_values))
    #     mae_ec.append(mean_absolute_error(pred_ec_values, gt_ec_values))
    #     mae_pc.append(mean_absolute_error(pred_pc_values, gt_pc_values))

    # # Compute average MAEs
    # avg_mae_bc = sum(mae_bc) / len(mae_bc)
    # avg_mae_ec = sum(mae_ec) / len(mae_ec)
    # avg_mae_pc = sum(mae_pc) / len(mae_pc)

    # vectorize and flatten
    pred_1d = np.concatenate(pred_1d, axis=0).flatten()
    gt_1d = np.concatenate(gt_1d, axis=0).flatten()

    mae = mean_absolute_error(pred_1d, gt_1d)
    pcc = pearsonr(pred_1d, gt_1d)[0]
    js_dis = jensenshannon(pred_1d, gt_1d)

    print("MAE: ", mae)
    print("PCC: ", pcc)
    print("Jensen-Shannon Distance: ", js_dis)
    # print("Average MAE betweenness centrality:", avg_mae_bc)
    # print("Average MAE eigenvector centrality:", avg_mae_ec)
    # print("Average MAE PageRank centrality:", avg_mae_pc)
    # return mae, pcc, js_dis, avg_mae_bc, avg_mae_ec, avg_mae_pc



In [ ]:
for i in range(args.splits):
    _, test_adjs, _, gt_matrices = data_fold_list[i]
    model = best_model_fold_list[i]
    model.eval()
    pred_matrices = np.zeros(gt_matrices.shape)
    with torch.no_grad():
        for j, test_adj in enumerate(test_adjs):
            pred_matrices[j] = model(torch.from_numpy(test_adj))[0].cpu()
    evaluate(pred_matrices, gt_matrices)

In [ ]:
A_HR_train = pd.read_csv("../data/hr_train.csv")

pca = PCA(n_components=0.99, whiten=False)
A_HR_train_pca = pca.fit_transform(A_HR_train)
print(A_HR_train_pca.shape)

gm = GaussianMixture(n_components=5, random_state=random_seed)
A_HR_train_label = gm.fit_predict(A_HR_train_pca)
unique, counts = np.unique(A_HR_train_label, return_counts=True)
print(np.asarray((unique, counts)).T)

X = np.load('A_LR_train_matrix.npy')
y = np.load('A_HR_train_matrix.npy')

n_sample = X.shape[0]
X_train, X_val, y_train, y_val = train_test_split(
    X.reshape(n_sample, -1), 
    y.reshape(n_sample, -1), 
    test_size=0.20, 
    random_state=random_seed,
    stratify=A_HR_train_label
)

X_train = X_train.reshape(-1, LR_size, LR_size)
X_val = X_val.reshape(-1, LR_size, LR_size)
y_train = y_train.reshape(-1, HR_size, HR_size)
y_val = y_val.reshape(-1, HR_size, HR_size)

print("Train size:", len(X_train))
print("Val size:", len(X_val))

netG = GSRNet(ks, args).to(device)
optimizerG = optim.Adam(netG.parameters(), lr=args.lr)
args.early_stop_patient = 5

netG = GSRNet(ks, args).to(device)
optimizerG = optim.Adam(netG.parameters(), lr=args.lr)

netD = Discriminator(args).to(device)
optimizerD = optim.Adam(netD.parameters(), lr=args.lr)

#GAN model
return_model = train_gan(
    netG, 
    optimizerG, 
    netD,
    optimizerD,
    X_train, 
    y_train, 
    args, 
    test_adj=X_val, 
    test_ground_truth=y_val,
    # stop_gan_mae=stop_gan_mae
)

In [ ]:
final_model = return_model

In [ ]:
print(args)

In [ ]:
final_model.eval()
pred_train_matrices = np.zeros(y_train.shape)
pred_val_matrices = np.zeros(y_val.shape)
with torch.no_grad():
    for j, test_adj in enumerate(X_train):
        pred_train_matrices[j] = final_model(torch.from_numpy(test_adj))[0].cpu()

    print("Train")
    evaluate(pred_train_matrices, y_train)

    for j, test_adj in enumerate(X_val):
        pred_val_matrices[j] = final_model(torch.from_numpy(test_adj))[0].cpu()

    print("Val")
    evaluate(pred_val_matrices, y_val)

In [ ]:
output_pred_list = []
final_model.eval()
with torch.no_grad():
    for i in range(A_LR_test_matrix.shape[0]):
        output_pred = final_model(torch.Tensor(A_LR_test_matrix[i]))[0].cpu()
        output_pred = MatrixVectorizer.vectorize(output_pred).tolist()
        output_pred_list.append(output_pred)

In [ ]:
output_pred_stack = np.stack(output_pred_list, axis=0)
output_pred_1d = output_pred_stack.flatten()
assert output_pred_1d.shape == (4007136, )

In [ ]:
df = pd.DataFrame({
    "ID": [i+1 for i in range(len(output_pred_1d))],
    "Predicted": output_pred_1d.tolist()
})

df

In [ ]:
df.to_csv("gsr_gat_residual_node_edge_drop_50perecentdrop_fold0.csv", index=False)